# Mine releases from selected projects

This notebook mine the releases

In [1]:
import os
import datetime
import json
import re
import traceback
import math

from multiprocessing import Pool
import pandas as pd

from ipywidgets import IntProgress
from IPython.display import display

import releasy

In [2]:
from util import (
    DATA_PATH,
    REPO_PATH,
    TMP_PATH, 
    CPU,
    CycleType,
    RAPID_RELEASE_LIM,
    TRAD_RELEASE_LIM,
    delta2days,
    is_rapid_release,
    is_trad_release,
    get_cycle,
    get_duration,
    get_start_delay,
    get_commits_in_advance
)

In [3]:
selected_projects = pd.read_csv(DATA_PATH / '10_projects_selected.csv')

## Mine Releases


In [5]:
def mine(name: str) -> releasy.Project:
    elapsed_time = datetime.datetime.now()
    
    try:
        repo_path = str(REPO_PATH / name)
        project = releasy.Miner(repo_path, name).apply(
            releasy.FinalReleaseMiner(),
            releasy.HistoryCommitMiner(),
            releasy.BaseReleaseMiner(),
            releasy.ContributorMiner(),
            releasy.SemanticReleaseMiner()
        ).mine()

        rapid_releases = [release for release in project.main_releases if is_rapid_release(release) and len(release.commits) >= 10]
        trad_releases = [release for release in project.main_releases if is_trad_release(release) and len(release.commits) >= 10]
        
        project_data = {
            'project': project.name,
            'prefixes': len(project.releases.prefixes()),
            'prefixes_names': str(" ".join(project.releases.prefixes())),
            'main_releases': len(project.main_releases),
            'rapid_releases': len(rapid_releases),
            'trad_releases': len(trad_releases),
            'patches': len(project.patches),
            'releases': len(project.releases)
        }
        
        release_data = []
        sreleases = [release for release in project.main_releases if len(release.commits) >= 10]
        for srelease in sreleases:
            # release = srelease.release
            # cycle_days = delta2days(get_cycle(srelease))
            # delay = delta2days(srelease.delay)
            # prev_release = srelease.prev_main_release

            if not srelease.commits or not srelease.prev_main_release:
                continue
                
            cycle_days = delta2days(get_cycle(srelease))
            duration = delta2days(get_duration(srelease))
            start_delay = delta2days(get_start_delay(srelease))
            commits_in_advance = get_commits_in_advance(srelease)
            
            release_data.append({
                'project': project.name,
                'release': srelease.name,
                'previous_release': srelease.prev_main_release.name if srelease.prev_main_release else '',
                'cycle_old': delta2days(srelease.cycle),
                'cycle_days': cycle_days,
                'rapid_release': is_rapid_release(srelease),
                'trad_release': is_trad_release(srelease),
                
                'commits': len(srelease.commits),
                'commits_in_advance': len(commits_in_advance),
                'start_delay': start_delay,
                'duration': duration,
                'patches': len(srelease.patches)     
            })
        
        elapsed_time = datetime.datetime.now() - elapsed_time
        project_data['time'] = elapsed_time
    except Exception as err:
        print(f"{name:40} {err=}")
        traceback.print_exception(err)
        project_data = {}
        release_data = []
    
    return project_data, release_data

SyntaxError: invalid syntax (3264603325.py, line 29)

In [ ]:
project_names = list(selected_projects['project'])
# project_names = project_names[:1]

In [ ]:
with Pool(processes=CPU) as pool:
    processed = 0
    project_results = []
    release_results = []
    progress = IntProgress(min=0, max=len(project_names))
    display(progress)
    for result in pool.imap_unordered(mine, project_names):
        project_result, release_result = result
        project_results.append(project_result)
        release_results.extend(release_result)
        progress.value += 1


In [ ]:
projects = pd.DataFrame(project_results)
projects.sample(1)

In [ ]:
releases = pd.DataFrame(release_results)
releases.sample(10)

In [ ]:
len(projects)

In [ ]:
len(releases)

In [ ]:
len(releases.query("rapid_release == True"))

In [ ]:
len(releases.query("trad_release == True"))

In [ ]:
len(releases.query("rapid_release == False and trad_release == False"))

In [ ]:
releases['commits'].sum()

In [ ]:
projects.to_csv(DATA_PATH / '20_projects.csv', index=False)

In [ ]:
releases.to_csv(DATA_PATH / '20_releases.csv', index=False)

In [ ]:
releases.query('abs(cycle_old - cycle_days) > 100')

In [ ]:
releases.query('(cycle_old - cycle_days) < -100')